In [1]:
import socket
from socket import timeout

In [2]:
serverName = 'localhost'

PortRecv = 15000
PortSend = 15001

BUFFER_SIZE = 60

recv = (serverName, PortRecv)
send = (serverName, PortSend)

recv_sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
send_sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

recv_sock.bind(recv)

In [3]:
output_file = 5  # some random assignment
seq_from = -1
buffer_acks = []

def make_packet(seq,ack,data):
    seq_field = (seq).to_bytes(4,byteorder='big')
    ack_field = (ack).to_bytes(4,byteorder='big')
    packet =  seq_field + ack_field + data
    return packet

def update_queue(recv_packets):
    global seq_from
    global output_file
    global buffer_acks
    if len(recv_packets)>0:
        buffer_acks += recv_packets
    buffer_acks = list(set(buffer_acks))
    buffer_acks.sort()
    last_ack = buffer_acks[-1][0]
    while len(buffer_acks)>0:
        if buffer_acks[0][0]<=seq_from:
            buffer_acks.pop(0)
        elif buffer_acks[0][0]==seq_from+1:
            output_file.write(buffer_acks[0][1])
            buffer_acks.pop(0)
            seq_from += 1
        else:
            break
    
    # making the neg acks into a packet
    neg_seqs = set(range(seq_from+1, last_ack+1))
    buffer_seqs = []
    for i in buffer_acks:
        buffer_seqs += [i[0]]
    buffer_seqs = set(buffer_seqs)
    neg_seqs -= buffer_seqs
    ans = [str(x) for x in neg_seqs]
    return ",".join(ans), last_ack

def upld(file_name):
    global seq_from
    global output_file
    global buffer_acks
    seq_from = -1
    output_file = open(file_name, "wb")
    last_ack_packet = None
    recv_packets = []
    buffer_acks = []
    transmission_complete = False
    while not transmission_complete or len(buffer_acks)>0:
        recv_packets = []
        recv_sock.settimeout(30) # timeout 1
        while True:
            try:
                print("waiting...")
                message, address = recv_sock.recvfrom(BUFFER_SIZE)
                seq = int.from_bytes(message[:4],byteorder='big')
                ack = int.from_bytes(message[4:8],byteorder='big')
                print(seq, ack)
                if ack==seq and seq >0:
                    transmission_complete = True
                recv_packets += [ (int.from_bytes(message[:4],byteorder='big'), message[8:]) ] # Storing (seq, data)
                print("recieved:{}".format(int.from_bytes(message[:4],byteorder='big')))
                break 
            except timeout:
                if last_ack_packet:
                    send_sock.sendto(last_ack_packet,send)
                if transmission_complete:
                    break
        recv_sock.settimeout(1) # different from prev timeout
        while True:
            try:
                message, address = recv_sock.recvfrom(BUFFER_SIZE)
                seq = int.from_bytes(message[:4],byteorder='big')
                ack = int.from_bytes(message[4:8],byteorder='big')
                print(seq, ack)
                if ack==seq and seq >0:
                    transmission_complete = True
                recv_packets += [ (int.from_bytes(message[:4],byteorder='big'), message[8:]) ]
                print("recieved:{}".format(int.from_bytes(message[:4],byteorder='big')))
                if transmission_complete:
                    break
            except timeout:
                break
        neg_seqs, last_ack = update_queue(recv_packets)
        last_ack_packet = make_packet(last_ack, last_ack, neg_seqs.encode())
        send_sock.sendto(last_ack_packet,send)
        print("Sent neg acks")
        print(transmission_complete)
        print(buffer_acks)
        #recv_sock.settimeout(1)
    output_file.close()
    print("Transfer Complete")

In [4]:
upld("rec.txt")

waiting...
0 122
recieved:0
1 122
recieved:1
2 122
recieved:2
3 122
recieved:3
4 122
recieved:4
5 122
recieved:5
Sent neg acks
False
[]
waiting...
6 122
recieved:6
7 122
recieved:7
8 122
recieved:8
Sent neg acks
False
[]
waiting...
9 122
recieved:9
10 122
recieved:10
11 122
recieved:11
Sent neg acks
False
[]
waiting...
12 122
recieved:12
13 122
recieved:13
14 122
recieved:14
Sent neg acks
False
[]
waiting...
15 122
recieved:15
16 122
recieved:16
17 122
recieved:17
Sent neg acks
False
[]
waiting...
18 122
recieved:18
19 122
recieved:19
20 122
recieved:20
Sent neg acks
False
[]
waiting...
21 122
recieved:21
22 122
recieved:22
23 122
recieved:23
Sent neg acks
False
[]
waiting...
24 122
recieved:24
25 122
recieved:25
26 122
recieved:26
Sent neg acks
False
[]
waiting...
27 122
recieved:27
28 122
recieved:28
29 122
recieved:29
Sent neg acks
False
[]
waiting...
30 122
recieved:30
31 122
recieved:31
32 122
recieved:32
Sent neg acks
False
[]
waiting...
33 122
recieved:33
34 122
recieved:34
35 1

In [5]:
output_file.close()